In [1]:
import re
from django.apps import apps
from django.db import models 


def forwards_index_ordering(apps):
    """Migration helper."""
    model_names = 'Aside', 'Pullquote', 'InlineHtml', 'StoryImage', 'StoryVideo'
    for model_name in model_names:
        Model = apps.get_model('stories', model_name)
        Model.objects.filter(top=True).update(placement='head')
        Model.objects.filter(index=None).update(placement=None)

forwards_index_ordering(apps)

FieldError: Cannot resolve keyword 'top' into field. Choices are: bodytext_html, bodytext_markup, created, id, index, modified, ordering, parent_story, parent_story_id, placement

In [2]:
TAGS = {
    'image': 'images',
    'quote': 'pullquotes',
    'box': 'asides',
    'video': 'videos',
    'html': 'inline_html_blocks',
}
FLAGS = {'<': 'left', '>': 'right', '=': 'wide'}

In [3]:
def find_tags(story, save=True):
    text = story.bodytext_markup
    group = "|".join(TAGS)
    regex = fr'^@(?P<tag>{group}): *(?P<content>.*?)$'
    matches = re.finditer(regex, text, flags=re.M | re.I)
    placements = {t: 0 for t in TAGS}
    print(f'\n{story}')
    
    top = story.images.top()
    if top.count() > 1:
        for i, img in enumerate(story.images.top(), 1): 
            img.ordering=i
            img.save()
            
    for m in matches:
        tag, content = m.groups()
        qs = getattr(story, TAGS[tag])
        indexes = [int(n) for n in re.findall(r'\d+', content)]
        flags = re.findall(r'[=><]', content)
        placements[tag] += 1
        n = placements[tag]
        placement_name = f'{tag}-{n}'
        flag = '' if not flags else f' | {FLAGS[flags[0]]}'
        placement_tag = f'[[ {placement_name}{flag} ]]'
        if not qs:
            placement_tag = ''
        for n, i in enumerate(indexes, 1):
            qs.filter(index=i).update(ordering=n, placement=placement_name)
        text = re.sub(m[0], placement_tag, text)
    
    for tag in TAGS.values():
        qs = getattr(story, tag)
        for item in qs.all():
            print(f'{item.ordering} {"-T"[item.top]} {item.placement:<8} {item}')
    
    if save:
        story.bodytext_markup = text
        story.save()

def update_placements():
    regex = f'@({"|".join(TAGS)}):'
    stories = Story.objects.filter(bodytext_markup__iregex=regex)
    for story in stories:
        find_tags(story, save=True)
    
update_placements()

In [4]:
for item in StoryImage.objects.order_by('parent_story_id', 'ordering'):
    print(f'{str(item)[:30]:<30}, {item.index} {item.ordering}, {item.placement:<10}, {item.parent_story_id}')

[eirik-welo-web.46488.jpg]    , 1 1, head      , 63553
[29-NETT-sno-EDD-04.45336.jpg], 1 1, head      , 63554
[27-NYH-nyhpolitistu-02-OD-2.3, 1 1, head      , 63555
[sj-018228.42040.jpg]         , 1 1, head      , 63556
[KHiO3.46258.jpg]             , 3 1, image-3   , 63557
[Khio4.46259.jpg]             , 4 1, image-4   , 63557
[KHiO1.46256.jpg]             , 1 1, image-1   , 63557
[KhIO2.46257.jpg]             , 2 1, image-2   , 63557
[KHiO5.46260.jpg]             , 5 1, image-5   , 63557
[khio6.46261.jpg]             , 6 6, auto      , 63557
[Khio7.46262.jpg]             , 7 7, head      , 63557
[Bilde1.46491.jpg]            , 1 1, head      , 63558
[cambridge.46157.jpg]         , 1 1, head      , 63559
[Sosialporn-5.46264.jpg]      , 1 1, head      , 63561
[ole-kristian-bratset.46402.jp, 1 1, head      , 63563
[digfv0485.46265.jpg]         , 1 1, head      , 63565
[Velferdstaag.46343.jpg]      , 1 1, head      , 63567
[Markus-Slettholm-2.46403.jpg], 2 2, auto      , 63567
[DSC7961.4

[LAERER.46661.jpg]            , 10 1, image-8   , 63716
[JUSS.46653.jpg]              , 2 1, image-10  , 63716
[WESTERDALS.46657.jpg]        , 6 1, image-2   , 63716
[KHIO.46659.jpg]              , 8 1, image-3   , 63716
[MUSIKK.46660.jpg]            , 9 1, image-6   , 63716
[SV.46658.jpg]                , 7 1, image-1   , 63716
[HF.46662.jpg]                , 11 11, head      , 63716
[06-NYH-politih-c-gskolen-EDD-, 1 1, head      , 63717
[06-NYH-IFI-SEKSUELL-TRAKASERI, 1 1, head      , 63718
[06-NYH-IFI-SEKSUELL-TRAKASERI, 2 2, auto      , 63718
[lesesaljuridiskfakultet-1-3-m, 1 1, head      , 63720
[29-NETT-sno-EDD-02.45334.jpg], 1 1, head      , 63721
[HKH-20180120-STP-OlgaDSC-9477, 8 1, image-2   , 63722
[HKH-20180119-STP-Demonstrasjo, 1 1, head      , 63722
[HKH-20180120-STP-OlgaDSC-9412, 3 1, image-3   , 63722
[HKH-20180123-MSK-Anton-DSC977, 7 1, image-6   , 63722
[HKH-20180118-STP-MaxDSC-8329., 4 1, image-5   , 63722
[HKH-20180119-STP-Demonstrasjo, 9 1, image-1   , 63722
[HKH-20

[lol4.47260.png]              , 4 3, image-1   , 63883
[009-NYH-AHO-EBJ-13-of-19.4748, 1 1, head      , 63885
[09-KUL-Young-Smul-aka-Samuel-, 3 1, image-1   , 63886
[09-KUL-Young-Smul-aka-Samuel-, 1 1, head      , 63886
[yung-smul-1.47463.jpg]       , 2 1, image-2   , 63886
[32-NYH-nyhosiblinde-08-PSS.39, 1 1, head      , 63890
[8-KUL-free-the-nipple-MC-1-1., 1 1, head      , 63891
[8-KUL-free-the-nipple-MC-2-3., 2 1, image-1   , 63891
[30-NYH-sveinstolen-MZ-04.4568, 2 1, image-1   , 63894
[09-NYH-LIKE-EBJ-22-of-37.4735, 1 1, head      , 63894
[AU147.47406.jpg]             , 1 1, head      , 63895
[08-NYH-KUTT-EBJ-1-of-12.47046, 1 1, head      , 63897
[youngstorget.46908.jpg]      , 1 1, head      , 63899
[KUL-Drikkepressfront-AN.36793, 1 1, head      , 63900
[Illustrasjon-Mikkrl-essay.479, 1 1, head      , 63901
[Juss.47515.jpg]              , 1 1, head      , 63903
[06-KUL-plastiskkirurgi-TWM-1., 3 1, image-2   , 63906
[06-KUL-plastiskkirurgi-TWM-4., 4 1, image-3   , 63906
[06-KUL-pl

[Tor-Mikkel-Wara.47989.jpg]   , 1 1, head      , 64046
[29-NYH-juridiskfakultet-EDD-0, 1 1, head      , 64053
[Nora-Geard-Nygaard-1.47505.jp, 1 1, head      , 64061
[Sunniva-Eide-Byline-01.47500., 2 2, auto      , 64061
[logo.48225.jpg]              , 1 1, head      , 64063
[Ingeborg-Nyrud-Byline-01.4726, 2 2, auto      , 64063
[Andrea-Kunz-Skrede-Byline-01., 3 3, auto      , 64063
[12-NYH-OsloBusinessForum-EBJ-, 1 1, head      , 64067
[12-NYH-OsloBusinessForum-EBJ-, 2 1, image-1   , 64067
[Hakon-Mikalsen-TSA-07.47971.j, 5 1, image-4   , 64068
[NSO-kandidater.48262.jpg]    , 1 1, head      , 64068
[12-NYH-Marte-Oien-er-ny-leder, 4 1, image-3   , 64068
[12-NYH-NSO-Ida-Austgulen-CAL-, 3 1, image-2   , 64068
[Kai-Steffen-Ostensen.48030.pn, 2 1, image-1   , 64068
[24-oye-oktoberfest-LLIF-nr01., 1 1, head      , 64070
[29792832-10160409620180372-70, 2 2, auto      , 64070
[Hans-Christian-Paulsen.44601., 1 1, head      , 64072
[screenshot.48228.png]        , 2 2, auto      , 64080
[johannes1